# <span style="color:#0080FF;">Udacity - Data Analyst Nanodegree </span>
## <span style="color:#B45F04;">Project 5: Identify Fraud from Enron Email</span>
### <span style="color:black;">Created by: Layne Newhouse</span>
### <span style="color:#6E6E6E;">Submitted: December X, 2016</span>

### *Background Information*

In 2000, Enron was one of the largest companies in the United States. By 2002, it had collapsed into bankruptcy due to widespread corporate fraud. In the resulting Federal investigation, a significant amount of typically confidential information entered into the public record, including tens of thousands of emails and detailed financial data for top executives. In this project, we will be playing detective by building a person of interest identifier based on financial and email data made public as a result of the Enron scandal. Labels will be provided via a hand-generated list of persons of interest in the fraud case, which means individuals who were indicted, reached a settlement or plea deal with the government, or testified in exchange for prosecution immunity. Features for the dataset include a subset of financial features as well as email features as follows:


<b>Financial Features:</b> ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees'] (all units are in US dollars)

<b>Email Features:</b> ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi'] (units are generally number of emails messages; notable exception is ‘email_address’, which is a text string)

<b>POI Label:</b> [‘poi’] (boolean, represented as integer)

### *Project Tasks*

1. Identify and remove outliers in the dataset 